In [1]:
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import os.path
import requests
import time
import json
from jsonpath_ng import jsonpath, parse
import urllib.parse
import csv

/Users/aiglesias/GitHub/solarchem-ontology


In [3]:
try:
    os.chdir(os.path.join(os.getcwd(), 'knowledge-graph')) 
    print(os.getcwd())
except:
    pass

/Users/aiglesias/GitHub/solarchem-ontology/knowledge-graph


# Construction, enrichment and evaluation of the Solarchem Knowledge Graph

## Mapping generation
Running Mapeathor to create the mapping file in YARRRML

In [6]:
!python -m mapeathor -i mappings/solarchem-mapping.xlsx -l yarrrml -o mappings/solarchem-mapping

Generating mapping file
Your mapping file is in mappings/solarchem-mapping.yml


## KG construction
Running Morph-KGC to construct the KG with the mapping generated in the previous step and the data in `./data`.

In [7]:
!python -m morph_kgc config.ini

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
INFO | 2024-05-13 18:03:23,548 | 122 mapping rules retrieved.
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_parser.py:485: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.rml_df = self.rml_df.replace(r'^\s*$', np.nan, regex=True)
INFO | 2024-05-13 18:03:23,625 | Mapping partition with 32 groups generated.
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_partitioner.py:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a

## KG validation
Quick validation to check that the KG is being generated with all the needed properties of the ontology.

In [40]:
g = rdflib.Graph()
g.parse("result/solarchem.nt")
print(len(g))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: could not convert string to float: '4 ,6 '
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: could not convert string to float: '0.5 + 1.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/aiglesias/miniconda3/envs/kgc/li

475783


### Articles

In [41]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX soa: <https://semopenalex.org/ontology/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article solar:hasExperimentExecution ?exp .
    }

""")

for row in q_res:
    print(f"Number of articles: {row['count'].value}")

Number of articles: 1093


In [ ]:
# old model (no soa)
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            bibo:doi ?doi ;
            bibo:abstract ?abs ;
            bibo:volume ?volume ;
            dc:title ?title ;
            bibo:issue ?issue ;
            dc:date ?date ;
            bibo:pageStart ?page .
    }

""")

for row in q_res:
    print(f"Number of articles with all properties: {row['count'].value}")

Number of articles with all properties: 415


In [ ]:
# old model (no soa)
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            solar:hasExperimentExecution ?expexec .
    }

""")

for row in q_res:
    print(f"Number of articles with experiments associated: {row['count'].value}")

Number of articles with experiments associated: 1096


### Experiment Executions

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?input) AS ?count)
    WHERE {
        ?expexec a solar:ExperimentExecution ;
            solar:reportedIn ?article ;
            prov:used ?input ;
            solar:hasCondition ?condition ;
            solar:hasLightSource ?lightsource .
    }

""")

for row in q_res:
    print(f"Number of experiments: {row['count'].value}")

Number of experiments: 18315


### Conditions

In [ ]:
## Time, pressure, pH, temperature
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?condition ?value ?unit
    WHERE {
        ?condition a solar:TimeCondition ; # solar:PressureCondition solar:pHCondition solar:TemperatureCondition
            qudt:numericValue ?value ;
            qudt:unit ?unit .
    } LIMIT 10
""")

for row in q_res:
    print(f"{row.condition} {row.value} {row.unit} ")

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT  (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition solar:SlurryReactor ; 
            #solar:hasCondition [ 
            #    a solar:ReactorVolumeCondition ;
            #    qudt:numericValue ?volume ] ;
            #solar:hasCondition [ 
            #    a solar:ResidenceTimeCondition ;
            #    qudt:numericValue ?restime ] ;
    } LIMIT 10
""")

for row in q_res:
    print(f"{row['count'].value}")

3258


In [ ]:
## Reactor medium and operation mode
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition solar:GasMedium ; # solar:GasMedium
            solar:hasCondition solar:BatchMode . # solar:ContinuousMode
    } 
""")

for row in q_res:
    print(row['count'].value)

2053


In [44]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
    PREFIX obi: <http://purl.obolibrary.org/obo/OBI_>

    SELECT DISTINCT (COUNT (DISTINCT ?lightSource) AS ?count)
    WHERE {
        ?lightSource a obi:0400065 ;
            solar:hasLamp ?lamp ;
            solar:hasWavelength ?wl ;
            solar:hasIrradiance [qudt:numericValue ?irradiance] ;
            solar:hasPower [qudt:numericValue ?power] ;
            solar:hasIlluminatedArea [qudt:numericValue ?illum_area] .
            
    } 
""")

for row in q_res:
    print(f"Number of lightsources with all properties: {row['count'].value}")

Number of lightsources with all properties: 165


### Inputs

In [72]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?input) AS ?count)
    WHERE {
        ?input a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solar:Catalyst ;
            qudt:numericValue ?value ;
            qudt:unit ?unit ;
            solar:hasBandGap ?bgap ;
            solar:hasSurfaceArea ?surfacearea .
    }

""")

for row in q_res:
    print(f"Number of catalyts with all properties: {row['count'].value}")

Number of catalyts with all properties: 6156


In [69]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?cocatalyst) AS ?count) 
    WHERE {
        ?cocatalyst a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solar:Co-catalyst ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of co-catalyts with all properties: {row['count'].value}")

Number of co-catalyts with all properties: 3176


In [73]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?support) AS ?count) 
    WHERE {
        ?support a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solarpc:Support ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of supports with all properties: {row['count'].value}")

Number of supports with all properties: 174


In [74]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?dopant) AS ?count) 
    WHERE {
        ?dopant a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solar:Dopant ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of dopants with all properties: {row['count'].value}")

Number of dopants with all properties: 541


In [75]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>

    SELECT (COUNT (DISTINCT ?dye) AS ?count) 
    WHERE {
        ?dye a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solarpc:Dye ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of dyes with all properties: {row['count'].value}")

Number of dyes with all properties: 33


## Experiment outputs generation

In [4]:
!python -m mapeathor -i mappings/outputs-solarchem-mapping.xlsx -l yarrrml -o mappings/outputs-solarchem-mapping

Generating mapping file
Your mapping file is in mappings/outputs-solarchem-mapping.yml


In [5]:
!python -m morph_kgc config-outputs.ini

INFO | 2024-06-20 12:54:32,603 | 334 mapping rules retrieved.
INFO | 2024-06-20 12:54:32,647 | Mapping partition with 324 groups generated.
INFO | 2024-06-20 12:54:32,647 | Maximum number of rules within mapping group: 2.
INFO | 2024-06-20 12:54:32,648 | Mappings processed in 1.827 seconds.
INFO | 2024-06-20 12:54:36,739 | Number of triples generated in total: 1837300.
INFO | 2024-06-20 12:54:36,739 | Materialization finished in 4.089 seconds.


In [6]:
g = rdflib.Graph()
g.parse("result/outputs-solarchem.nt")
print(len(g))

1837300


In [21]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <https://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
    PREFIX prov: <http://www.w3.org/ns/prov#>

    SELECT ?output ?label ?value ?unit ?expexec
    WHERE {
        ?output a solar:Output ;
            rdfs:label ?label ;
            qudt:numericValue ?value ;
            qudt:unit ?unit ;
            prov:wasGeneratedBy ?expexec .
    }

""")

for row in q_res:
    print(f"{row.output} {row.label} {row.value} {row.unit} {row.expexec}")

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <https://w3id.org/solar/o/core#>
    PREFIX solarpc: <https://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
    PREFIX prov: <http://www.w3.org/ns/prov#>

    SELECT (COUNT (DISTINCT ?output) AS ?count)
    WHERE {
        ?output a solar:Output ;
            rdfs:label ?label ;
            qudt:numericValue ?value ;
            qudt:unit ?unit ;
            prov:wasGeneratedBy ?expexec .
    }

""")

for row in q_res:
    print(f"Number of outputs with all properties: {row['count'].value}")

Number of outputs with all properties: 18028


## Merging KGs into one

In [18]:
!cat result/solarchem.nt result/outputs-solarchem.nt | uniq > result/solarchem-kg.nt

## Finding chemical concepts in CHEBI and Wikidata

### Create a list of unique values of chemicals mentioned in the data

In [30]:
exp_df = pd.read_csv("data/catalystsdata.csv")
exp_df.head()

,ID,No_de_Ref,Catalyst_name,TiO2_crystal_structure,Catalyst,Support,support_percent,Co_catalyst,co_catalyst_2,co_catalyst_3,...,C4H10_mol_m2h,C5H10_mol_m2h,C5H12_mol_m2h,CH3OH_mol_m2h,C2H5OH_mol_m2h,CH3COH_mol_m2h,HCOOH_mol_m2h,CH2O_mol_m2h,C2H4O2_mol_m2h,timestamp
0,1,1,TiO2,Anatase,TiO2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,0000-00-00 00:00:00
1,2,1,TiO2,Anatase,TiO2,NaN,NaN,Ag,NaN,NaN,...,NaN,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,0000-00-00 00:00:00
2,3,1,Ag/TiO2,Anatase,TiO2,NaN,NaN,Ag,NaN,NaN,...,NaN,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,0000-00-00 00:00:00
3,4,1,Ag/TiO2,Anatase,TiO2,NaN,NaN,Ag,NaN,NaN,...,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,0000-00-00 00:00:00
4,5,1,Ag/TiO2,Anatase,TiO2,NaN,NaN,Ag,NaN,NaN,...,NaN,NaN,NaN,0.03,NaN,NaN,NaN,NaN,NaN,0000-00-00 00:00:00


List of output formulas that appear as columns

In [31]:
outputs = pd.Series([
    'H2',
    'CO',
    'CH4',
    'C2H4',
    'C2H6',
    'C3H6',
    'C3H8',
    'C4H8',
    'C4H10',
    'C5H10',
    'C5H12',
    'CH3OH',
    'C2H5OH',
    'CH3COH',
    'HCOOH',
    'CH2O',
    'C2H4O2'
])

In [32]:
chems_df = pd.DataFrame()
chems_df['Chemicals'] = pd.concat([exp_df['Catalyst'], exp_df['Support'], exp_df['Co_catalyst'], exp_df['co_catalyst_2'], exp_df['co_catalyst_3'], exp_df['Dopant'], exp_df['Dyes'], exp_df['Reductant'],outputs]).unique()
chems_df

,Chemicals
0,TiO2
1,ZrO2
2,Carbon
3,b-Ga2O3
4,Zn2GeO4
...,...
1052,C5H12
1053,C2H5OH
1054,CH3COH
1055,CH2O


### Find correspondences of mentions with CHEBI via EBI API
https://www.ebi.ac.uk/ebisearch/documentation/rest-api

In [20]:
chems_df = pd.read_csv("data/chemicals.csv", dtype={'chebi': str})
chems_df

,Chemicals,chebi,wikidata,wikidata_multiple,chebi_multiple,uri
0,TiO2,32234.0,Q193521,NaN,NaN,http://www.wikidata.org/entity/Q193521
1,ZrO2,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/ZrO2
2,Carbon,NaN,NaN,NaN,"23012,50814,50813,36928,23014,30253,149691,294...",http://w3id.org/solar/i/Chemical/Carbon
3,b-Ga2O3,NaN,NaN,NaN,"59216,36586,22702,50860",http://w3id.org/solar/i/Chemical/b-Ga2O3
4,Zn2GeO4,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/Zn2GeO4
...,...,...,...,...,...,...
1039,KHCO3 and H2SO4,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/KHCO3+and+H2SO4
1040,H2O:TEOA,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/H2O%3ATEOA
1041,Ethyl Acetate EA,NaN,NaN,NaN,"26347,23849,33853",http://w3id.org/solar/i/Chemical/Ethyl+Acetate+EA
1042,TEOA:MeCN 1:4,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/TEOA%3AMeCN+1...


In [33]:
headers = {'Accept': 'application/json'}
query = 'https://www.ebi.ac.uk/ebisearch/ws/rest/chebi?query={}'
jsonpath = parse('$.entries[*].id')

chebi_id = {}
chebi_multiple = {}
for label in chems_df['Chemicals']:
    
    response = requests.get(query.format(label), headers=headers)
    
    if response.status_code == 200:
        if len(response.json()['entries']) == 0:
            chebi_id[label] = ''
        elif len(response.json()['entries']) == 1:
            chebi_id[label] = str(response.json()['entries'][0]['id'])
            print(chebi_id[label])
        else:
            res_list = [match.value for match in jsonpath.find(response.json())]
            chebi_multiple[label] = ','.join(res_list)
            print(chebi_multiple[label])
    else:
            chebi_id[label] = ''
    


32234
23012,50814,50813,36928,23014,30253,149691,29436,35605,36927,36929,36930,50809,35604,139538
59216,36586,22702,50860
31652,63616,63722,51867,32877,35358,25212
50836
30187,30191,30192
31344
31794
81908
29390
36973,134404,132889,50594,33418,50797,50596,50595,36977,50803,33415,46730,132888,50860
36973,132889,132888,33418,50860
79089
50832,50833
30341
133333
27363,30185,37254,37255,29105,133337,88218,133340,49976,36578
36973,132889,132888,33418,50860
85287,90970,2341,142482,140626,140627,90980,136820,142483,142024,77193,77196,21345,21412,143102
27573,30584,30582,30586,30585,48150,33478,59830,37171,53393,77023,84275,37172,48139,30581
88916,88729,89101,89172,89179,89292,89293,89294,89500,89534,89535,89544,89547,89560,89692
51110,51113,51115
67195,62488,17138,42626
50818,50819,50824,139548
64132,64131,64123,45744,82851,9423,33517,48279,46848,46845,48367,25697,35274
50836
32234
137381,137373,137379
30399,28112,49786,137381,144805,85620,137373,47739,177874,133340,144804,25516,61258,137379,

In [34]:
chems_df['chebi'] = chems_df['Chemicals'].map(chebi_id)
chems_df['chebi_multiple'] = chems_df['Chemicals'].map(chebi_multiple)
chems_df

,Chemicals,chebi,chebi_multiple
0,TiO2,32234,NaN
1,ZrO2,,NaN
2,Carbon,NaN,"23012,50814,50813,36928,23014,30253,149691,294..."
3,b-Ga2O3,NaN,"59216,36586,22702,50860"
4,Zn2GeO4,,NaN
...,...,...,...
1052,C5H12,NaN,"30358,30362"
1053,C2H5OH,16236,NaN
1054,CH3COH,,NaN
1055,CH2O,79073,NaN


In [35]:
chems_df.to_csv("data/chemicals.csv", index=False, sep=",")

### Find corresponences with Wikidata via SPARQL endpoint

In [36]:
chems_df = pd.read_csv("data/chemicals.csv", dtype={'chebi': str})
chems_df

,Chemicals,chebi,chebi_multiple
0,TiO2,32234,NaN
1,ZrO2,NaN,NaN
2,Carbon,NaN,"23012,50814,50813,36928,23014,30253,149691,294..."
3,b-Ga2O3,NaN,"59216,36586,22702,50860"
4,Zn2GeO4,NaN,NaN
...,...,...,...
1052,C5H12,NaN,"30358,30362"
1053,C2H5OH,16236,NaN
1054,CH3COH,NaN,NaN
1055,CH2O,79073,NaN


In [37]:
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

query = """
    SELECT DISTINCT ?chem ?chebi_wd
    WHERE
    {{
      ?chem wdt:P683 ?chebi_wd ;
            skos:altLabel "{}"@en .
    }} 
"""

jsonpath = parse('$.results.bindings[*].chem.value')

In [38]:
wd_id = {}
wd_multiple = {}
for label in chems_df['Chemicals']:
    
    time.sleep(0.5)
    wd_id[label] = ''
    sparql.setQuery(query.format(label))
    
    try:
        res = sparql.queryAndConvert()
        
        if len(res['results']['bindings']) == 0:
            wd_id[label] = ''
        elif len(res['results']['bindings']) == 1:
            wd_id[label] = res['results']['bindings'][0]['chem']['value'].replace('http://www.wikidata.org/entity/', '')
            print(wd_id[label])
        else:
            js_res = jsonpath.find(res)
            res_list = [match.value for match in jsonpath.find(res)]
            wd_multiple[label] = ','.join(res_list).replace('http://www.wikidata.org/entity/', '')
            print(wd_multiple[label])
            
    except:
        wd_id[label] = ''

Q193521
Q190077
Q214769
Q412356
Q216918
Q758
Q670
Q205021
Q744
Q186357
Q424257
Q897
Q880
Q27104633,Q740
Q116269,Q43010
Q428946
Q716
Q753
Q425127
Q409173
Q2380192
Q407674
Q186474,Q27089394,Q623
Q56440222
Q423673
Q1090
Q1087
Q1086
Q1089
Q26841311,Q725
Q1091
Q102769
Q407849
Q414860
Q411214
Q2314
Q183290,Q682
Q414806
Q421857
Q1801
Q1385
Q1038
Q677
Q27144155
Q1053
Q204714
Q407867
Q27158631,Q942
Q731
Q708
Q484583,Q20035886,Q674
Q484940,Q1103
Q72499282
Q688
Q314036
Q27104698,Q663
Q1963030
Q207442
Q938
Q660
Q618
Q1112
Q4847704
Q706
Q185906,Q29519883,Q627
Q568
Q1094
Q629
Q27143862
Q170545,Q27103475
Q1849
Q428946
Q483752,Q722
Q1819
Q27104698
Q895
Q243547,Q20816880,Q703
Q658
Q737
Q27144218,Q1832
Q485277,Q60168
Q315860
Q207843
Q879
Q1046
Q1096
Q2370426
Q209143
Q283
Q37129
Q161233
Q3027893
Q16392
Q410529
Q339975
Q179731
Q407806
Q211086
Q14982
Q410387,Q617563
Q16392
Q423282
Q192642
Q408022,Q27098592
Q14982
Q410728
Q407895
Q4118
Q424314
Q379885
Q407577
Q27144175
Q2025
Q153


In [39]:
outputs_wd = {
    'H2': 'Q3027893',
    'CO': 'Q2025',
    'CH4': 'Q37129',
    'C2H4': 'Q151313',
    'C2H6': 'Q52858',
    'C3H6': 'Q151324',
    'C3H8': 'Q131189',
    'C4H8': 'Q151335',
    'C4H10': 'Q134192',
    'C5H10': 'Q151353',
    'C5H12': 'Q2799651',
    'CH3OH': 'Q14982',
    'C2H5OH': 'Q153',
    'CH3COH': 'Q61457',
    'HCOOH': 'Q161233',
    'CH2O': 'Q161210',
    'C2H4O2': 'Q47512',
}
wd_id.update(outputs_wd)

In [40]:
chems_df['wikidata'] = chems_df['Chemicals'].map(wd_id)
chems_df['wikidata_multiple'] = chems_df['Chemicals'].map(wd_multiple)
chems_df

,Chemicals,chebi,chebi_multiple,wikidata,wikidata_multiple
0,TiO2,32234,NaN,Q193521,NaN
1,ZrO2,NaN,NaN,,NaN
2,Carbon,NaN,"23012,50814,50813,36928,23014,30253,149691,294...",,NaN
3,b-Ga2O3,NaN,"59216,36586,22702,50860",,NaN
4,Zn2GeO4,NaN,NaN,,NaN
...,...,...,...,...,...
1052,C5H12,NaN,"30358,30362",Q2799651,NaN
1053,C2H5OH,16236,NaN,Q153,NaN
1054,CH3COH,NaN,NaN,Q61457,NaN
1055,CH2O,79073,NaN,Q161210,NaN


In [41]:
chems_df.to_csv("data/chemicals.csv", index=False, sep=",")

### Creating subject URI

In [42]:
chems_df = pd.read_csv("data/chemicals.csv", dtype={'chebi': str})
chems_df

,Chemicals,chebi,chebi_multiple,wikidata,wikidata_multiple
0,TiO2,32234,NaN,Q193521,NaN
1,ZrO2,NaN,NaN,NaN,NaN
2,Carbon,NaN,"23012,50814,50813,36928,23014,30253,149691,294...",NaN,NaN
3,b-Ga2O3,NaN,"59216,36586,22702,50860",NaN,NaN
4,Zn2GeO4,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1052,C5H12,NaN,"30358,30362",Q2799651,NaN
1053,C2H5OH,16236,NaN,Q153,NaN
1054,CH3COH,NaN,NaN,Q61457,NaN
1055,CH2O,79073,NaN,Q161210,NaN


In [43]:
uri = {}
for index, row in chems_df.iterrows():
    if not pd.isna(row['wikidata']):
        uri[row['Chemicals']] = 'http://www.wikidata.org/entity/' + row['wikidata']
    elif not pd.isna(row['chebi']):
        uri[row['Chemicals']] = 'http://purl.obolibrary.org/obo/CHEBI_' + str(row['chebi'])
    else:
        uri[row['Chemicals']] = 'http://w3id.org/solar/i/Chemical/' + urllib.parse.quote_plus(str(row['Chemicals']))

chems_df['uri'] = chems_df['Chemicals'].map(uri)
chems_df

,Chemicals,chebi,chebi_multiple,wikidata,wikidata_multiple,uri
0,TiO2,32234,NaN,Q193521,NaN,http://www.wikidata.org/entity/Q193521
1,ZrO2,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/ZrO2
2,Carbon,NaN,"23012,50814,50813,36928,23014,30253,149691,294...",NaN,NaN,http://w3id.org/solar/i/Chemical/Carbon
3,b-Ga2O3,NaN,"59216,36586,22702,50860",NaN,NaN,http://w3id.org/solar/i/Chemical/b-Ga2O3
4,Zn2GeO4,NaN,NaN,NaN,NaN,http://w3id.org/solar/i/Chemical/Zn2GeO4
...,...,...,...,...,...,...
1052,C5H12,NaN,"30358,30362",Q2799651,NaN,http://www.wikidata.org/entity/Q2799651
1053,C2H5OH,16236,NaN,Q153,NaN,http://www.wikidata.org/entity/Q153
1054,CH3COH,NaN,NaN,Q61457,NaN,http://www.wikidata.org/entity/Q61457
1055,CH2O,79073,NaN,Q161210,NaN,http://www.wikidata.org/entity/Q161210


In [44]:
chems_df.to_csv("data/chemicals.csv", index=False, sep=",")